In [11]:
import os
import re
import laspy
import rasterio
import numpy as np

def normalize_las(las_filename, dtm_filename, output_filename):
    # Load LAS file
    las_file = laspy.read(las_filename)

    # Load DTM raster file
    with rasterio.open(dtm_filename) as dtm:
        # Sample DTM values at LAS points
        dtm_elevations = np.array(list(rasterio.sample.sample_gen(dtm, zip(las_file.x, las_file.y))))

    # Check if the number of LAS points matches the number of sampled DTM values
    if len(las_file.z) != len(dtm_elevations):
        raise ValueError("Number of LAS points does not match the number of sampled DTM values.")

    # Normalize LAS points based on DTM elevation
    normalized_elevations = np.array(las_file.z) - dtm_elevations.flatten()

    # Update LAS file with normalized elevations
    las_file.z = normalized_elevations.tolist()  # Convert back to list for laspy compatibility

    # Save the normalized LAS file
    las_file.write(output_filename)

def process_folders(input_folder, output_folder):
    # Traverse the input folder and its subfolders
    for root, dirs, files in os.walk(input_folder):
        # Filter for LAS and TIFF files
        las_files = [f for f in files if f.lower().endswith(".las")]
        tiff_files = [f for f in files if f.lower().endswith(".tif")]

        # Check if there is exactly one LAS file and one TIFF file
        if len(las_files) == 1 and len(tiff_files) == 1:
            # Construct the file paths for LAS and DTM files
            las_filename = os.path.join(root, las_files[0])
            dtm_filename = os.path.join(root, tiff_files[0])

            # Construct the output file path
            relative_path = os.path.relpath(las_filename, input_folder)
            output_filename = os.path.join(output_folder, relative_path)

            # Create the output folder if it doesn't exist
            os.makedirs(os.path.dirname(output_filename), exist_ok=True)

            # Process the LAS file
            normalize_las(las_filename, dtm_filename, output_filename)

if __name__ == "__main__":
    input_folder = r"input"
    output_folder = r"output"

    process_folders(input_folder, output_folder)
